In [1]:
import requests
import pandas as pd
import numpy as np
from keys import *

#Nombre de la Ciudad 
city = "Naucalpan"

#Abreviatura de del PAIS
country = "MX"

#Se pide el clima a la página web/
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

In [2]:
#Guardam los datos del clima en formato JSON
data = response.json()
#print(data)

# Extraemos la lista del pronóstico del clima
forecast_list = data.get('list', [])

# Preparamos listas para las columnas del DataFrame
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

# extraemos los datos para cada columna, tamperatura, humedad, etc 

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Creamos DataFrame
df = pd.DataFrame({
    'time': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind_speed': wind_speeds,
    'rain_volume_3h': rain_volumes,
    'snow_volume_3h': snow_volumes
})

print(df.head())

                  time  temperature  humidity weather_status  wind_speed  \
0  2025-10-02 00:00:00        23.41        36         Clouds        5.83   
1  2025-10-02 03:00:00        21.27        44         Clouds        3.23   
2  2025-10-02 06:00:00        18.08        62         Clouds        2.46   
3  2025-10-02 09:00:00        14.92        77           Rain        2.52   
4  2025-10-02 12:00:00        14.69        75           Rain        1.69   

   rain_volume_3h  snow_volume_3h  
0             NaN             NaN  
1             NaN             NaN  
2             NaN             NaN  
3            0.12             NaN  
4            0.17             NaN  
